# JS loops

## Symbol.iterator

https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Symbol/iterator

In [10]:
const iterable1 = {};


iterable1[Symbol.iterator] = function* () {
  yield 'Tom';
  yield 'John';
  yield 'Arcady';
  yield 'Benjamin';
};

// @ts-ignore
console.log([...iterable1]);

[ 'Tom', 'John', 'Arcady', 'Benjamin' ]


In [23]:
class Foo {
  *[Symbol.iterator]() {
    yield 1;
    yield 2;
    yield 3;
  }
}

const someObj = {
  *[Symbol.iterator]() {
    yield "a";
    yield "b";
  },
};

console.log(...new Foo()); // 1, 2, 3
console.log(...someObj); // 'a', 'b'

for (let v of someObj) {
  console.log(v);
}

1 2 3
a b
a
b


Implementation without `*/yield` with `return object: {next() /*...*/}`

In [1]:
const collection = {
  one: 1,
  two: 2,
  three: 3,
  [Symbol.iterator]() {
    const values = Object.keys(this);
    let i = 0;
    return {
      next: () => {
        return {
          value: this[values[i++]],
          done: i > values.length,
        };
      },
    };
  },
};

const iterator = collection[Symbol.iterator]();

console.log(iterator.next()); // → {value: 1, done: false}
console.log(iterator.next()); // → {value: 2, done: false}
console.log(iterator.next()); // → {value: 3, done: false}
console.log(iterator.next()); // → {value: undefined, done: true}

{ value: 1, done: false }
{ value: 2, done: false }
{ value: 3, done: false }
{ value: undefined, done: true }


## Using array default Symbol.iterator

- https://github.com/sudheerj/javascript-interview-questions#how-does-synchronous-iteration-works

In [3]:
const iterable = ["one", "two", "three"];
const iterator = iterable[Symbol.iterator]();
console.log(iterator.next()); // { value: 'one', done: false }
console.log(iterator.next()); // { value: 'two', done: false }
console.log(iterator.next()); // { value: 'three', done: false }
console.log(iterator.next()); // { value: 'undefined, done: true }

{ value: 'one', done: false }
{ value: 'two', done: false }
{ value: 'three', done: false }
{ value: undefined, done: true }


## What for is for...of statement

Iterating over iterable objects or elements such as built-in String, Array, Array-like objects (like arguments or NodeList), TypedArray, Map, Set, and user-defined iterables. Objects can't be iterated.

In [6]:
var arrayIterable = [10, 20, 30, 40, 50];

for (let value of arrayIterable) {
  console.log(value)
}

10
20
30
40
50


In [14]:
var map = new Map();

map.set('b', 'Bob');
map.set('a', 'Alice');

for (let v of map) {
  console.log(v);
}

[ 'b', 'Bob' ]
[ 'a', 'Alice' ]


In [27]:
var obj = {b: 'Bob', a: 'Alice'};


// objects can't be iterated directly
for (let v of obj) {}

5:15 - Type '{ b: string; a: string; }' must have a '[Symbol.iterator]()' method that returns an iterator.


In [26]:
for (let v of Object.values(obj)) {
  console.log(v);
}

Bob
Alice


In [19]:
function foo() {
  for (const value of arguments) {
    console.log(value);
  }
}

foo(1, 2, 3);

1
2
3


## Symbol.asyncIterator

- https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Symbol/asyncIterator

In [22]:
const delayedResponses = {
  delays: [500, 1300, 3500],

  wait(delay) {
    return new Promise((resolve) => {
      setTimeout(resolve, delay);
    });
  },

  async *[Symbol.asyncIterator]() {
    for (const delay of this.delays) {
      await this.wait(delay);
      yield `Delayed response for ${delay} milliseconds`;
    }
  },
};

(async () => {
  for await (const response of delayedResponses) {
    console.log(response);
  }
})();

Promise { <pending> }
Delayed response for 500 milliseconds
Delayed response for 1300 milliseconds
Delayed response for 3500 milliseconds


## Difference between for..in and for..of

- `for...in` iterates over the enumerable string properties of an object.
- `for...of` iterates over values that the iterable object defines to be iterated over with `Symbol.iterator`.

- https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Statements/for...of#difference_between_for...of_and_for...in

In [33]:
const iterable = [3, 5, 7];

// @ts-ignore
iterable.foo = "hello";

// @ts-ignore
iterable.bar = () => {};

for (const i in iterable) {
  console.log(i);
}

0
1
2
foo
bar


In [34]:
for (const i of iterable) {
  console.log(i);
}

3
5
7


In [35]:
// "fixing" for..in
for (const i in iterable) {
  if (iterable.hasOwnProperty(i)) {
    console.log(i);
  }
}

0
1
2
foo
bar


## Generators

https://github.com/sudheerj/javascript-interview-questions#46-what-is-the-output-of-below-code

A return statement in a generator function will make the generator finish. 

When a generator is finished, subsequent `next()` calls will always return `{ value: undefined }`.

In [4]:
function* yieldAndReturn() {
  yield 1;
  return 2;
  yield 3;
}

var myGenObj = yieldAndReturn();
console.log(myGenObj.next());
console.log(myGenObj.next());
console.log(myGenObj.next());

{ value: 1, done: false }
{ value: 2, done: true }
{ value: undefined, done: true }


https://github.com/sudheerj/javascript-interview-questions#47-what-is-the-output-of-below-code

Upon exiting a loop(on completion or using break & return), the generator is closed and trying to iterate over it again does not yield any more results.

In [5]:
const myGenerator = (function* () {
  yield 1;
  yield 2;
  yield 3;
})();

for (const value of myGenerator) {
  console.log(value);
  break;
}

for (const value of myGenerator) {
  console.log(value);
}

1
